## Cluster analysis
### Objectives
* Classify nonsense and frameshift mutations
* Filter based on CADD > 20, Combined Annotation Dependent Depletion (CADD) method
* Identify likely deleterious variant clusters in patient genes
* Generate statistics on gene clustering

### Description
Given a large set of cases, variants contributing to a disease trait will form detectable clusters within specific regions of a gene or genes which, when mutated, lead to the disease. Given reliable sequencing data, clusters can be uncovered using the Wd statistic.

For each gene coding sequence (CDS), we enter the number of hits that fall within the CDS and the probabibility of a single hit falling within this particular CDS given the exome's total CDS size.

### Resources used in the original analysis
* [Combined Annotation Dependent Depletion](https://cadd.gs.washington.edu/) (CADD) [v.1.3](https://cadd.gs.washington.edu/static/ReleaseNotes_CADD_v1.3.pdf)
* [googleCloudStorageR](https://github.com/cloudyr/googleCloudStorageR) v.0.4.0
* [Ensembl 75](http://feb2014.archive.ensembl.org/index.html) provides gene coding sequence sizes

### Analysis
| Steps                                      | Input                                           | Output |
|--------------------------------------------|-------------------------------------------------|--------|
| Filter CADD >20 | gemini_out.txt                 | gemini_filtered     |
| Count variants per gene                    |gemini_filtered                                  | variants_per_gene   |
| Add genes that have a variant count of 0 or X | variants_per_gene | variants_per_CDS |
| Sort and add CDS length as fraction of total CDS length | variants_per_CDS | variants_per_CDS_withsizes
| Cluster analysis                           | variants_per_CDS_withsizes | new_data     |



**Section 1: Install google cloud storage for R package in order to access the WDL outputs from Gemini.**

Install google cloud's R package and generate a key to access the workspace's google bucket.

In [1]:
install.packages("googleCloudStorageR")
key <- Ronaldo::getServiceAccountKey()

Installing package into ‘/home/jupyter-user/.rpackages’
(as ‘lib’ is unspecified)


Save the key and write it to a json file that can be accessed from your local cluster.

In [2]:
fileConn<-file("/home/jupyter-user/key.json")
writeLines(key, fileConn)
close(fileConn)

Set environment variable so that googleCloudStorageR can authenticate itself.

In [3]:
Sys.setenv("GCS_AUTH_FILE" = "/home/jupyter-user/key.json")

Load package googleCloudStorageR. To see more features use *library(help = googleCloudStorageR)*.

In [4]:
library(googleCloudStorageR)

Setting scopes to https://www.googleapis.com/auth/devstorage.full_control
Successfully authenticated via /home/jupyter-user/key.json


**Section 2: Read the output from Gemini and start filtering. We are filtering out any variants that are not stop mutations or frameshift; since they are likely deleterious, or missense. For missense, we further focus on variants predicted to be highly deleterious using CADD (i.e. CADD score >= 20).**

Note: In the workshop we will be accessing precomputed files since the method will not complete in time. For reproducibility, the author would adjust the WDL to output a file to the same location every time or have a copy of the Gemini output pushed to a separate location. Right now every new submission completed is stored in a separate directory for provenance purposes. 

Read gemini_out.txt from the Google bucket and save it to the local Jupyter cluster.

In [5]:
gcs_get_object("gs://firecloud-workshops/181017-ashg18/predicted-effects/Test-cohort-A100.jointcalls.filtered.gemini-predictions.txt",saveToDisk = "../gemini_out.txt",overwrite= TRUE)
gemini_out<-read.table("/home/jupyter-user/gemini_out.txt",sep = "\t", header = TRUE)

2018-10-14 18:40:25 -- Saved 181017-ashg18/predicted-effects/Test-cohort-A100.jointcalls.filtered.gemini-predictions.txt to ../gemini_out.txt (3 Mb)


[1] TRUE

Replace 'None' with 0 for the next step where we filter on a numeric CADD score.

In [6]:
gemini_out$cadd_scaled <- as.numeric(as.character(gemini_out$cadd_scaled))
gemini_out$cadd_scaled[is.na(gemini_out$cadd_scaled)] <- 0

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

Filter for CADD score >= 20.

In [7]:
gemini_filtered<-gemini_out[(gemini_out$biotype=="protein_coding")& (gemini_out$cadd_scaled>=20),]

Read Ensembl gene CDS sizes from a file in the Google bucket.

In [8]:
gcs_get_object("gs://firecloud-workshops/181017-ashg18/annotation/Ensembl75_gene_CDS_sizes.txt",saveToDisk = "../ENS75.txt",overwrite= TRUE)
CDS_size <- read.table("/home/jupyter-user/ENS75.txt",sep = "\t", header = TRUE)

2018-10-14 18:40:55 -- Saved 181017-ashg18/annotation/Ensembl75_gene_CDS_sizes.txt to ../ENS75.txt (316.1 Kb)


[1] TRUE

Count number of variants per gene.

In [9]:
variants_per_gene<-aggregate(x = gemini_filtered$gene, by = list(gene = gemini_filtered$gene), FUN = length)
colnames(variants_per_gene) <- c("gene","variant_count")

Add genes that have a variant count of 0 and remove genes that don't have an incomplete CDS.

In [10]:
variants_per_CDS<-rbind.data.frame(variants_per_gene[variants_per_gene$gene %in% CDS_size$gene,], data.frame(gene = setdiff(CDS_size$gene,variants_per_gene$gene), variant_count = 0))                   


Sort and add CDS length as fraction of total CDS length.

In [11]:
sorted<-variants_per_CDS[order(as.character(variants_per_CDS$gene)),]
variants_per_CDS_withsizes<-data.frame(sorted,length_proportion=CDS_size$length_proportion)
variants_per_CDS_withsizes

,gene,variant_count,length_proportion
13507,A1BG,0,0.000033100
21000,A1CF,0,0.000132969
1,A2M,3,0.000069400
2,A2ML1,1,0.000075600
31000,A3GALT2,0,0.000014700
3,A4GALT,1,0.000033500
41000,A4GNT,0,0.000025500
4,AAAS,1,0.000028900
5,AACS,1,0.000095800
51000,AADAC,0,0.000023600


**Section 3: Perform the Cluster.Test method to identify genes where significantly more variants are observed than expected.
The method takes in two arguments: NbrInTarget and TargetProbability.**

**NbrInTarget** is derived from our data and consists of a list of numbers which correspond to the number of variants (or hits) per each CDS.

**TargetProbability** is the fraction of the total human coding sequence represented by each gene coding sequence. (i.e. gene CDS size/total CDS size). This information is typically derived from BioMart but we can just provide it directly in a file in the Google bucket.

Therefore:
* N is the number of genes/gene CDS being tested.
* TotalHits is the total number of variants.
* ExpectedHits is the number of variants we expect to see in a gene CDS based on its size.

The last line is applying multiple testing correction to the statistical method i.e. N tests for N gene CDS.


In [12]:
Cluster.Test<-function(NbrInTarget,TargetProbability){
N<-length(NbrInTarget)
TotalHits<-sum(NbrInTarget)
ExpectedHits<-TotalHits*TargetProbability
# N*ppois(NbrInTarget,ExpectedHits,lower=F)
1-exp(ppois(NbrInTarget,ExpectedHits,lower=T,log=T)*N)
}

**Section 4: Review the output of the Cluster.Test.**

This table tells us that NOTCH1 is the locus harbouring the highest excess of rare, deleterious variants among ToF patients. The idea is that we are comparing the number of variants we would expect by chance and the number of variants we actually observe. Some genes are so big that they’ll definitely have more of variants than NOTCH1, but what’s significant for NOTCH1 is how many variants it has given its size.

In [13]:
newdata <- data.frame(variants_per_CDS_withsizes, cluster.test.result = Cluster.Test(variants_per_CDS_withsizes$variant_count,variants_per_CDS_withsizes$length_proportion))
newdata[order(newdata$cluster.test.result),]

,gene,variant_count,length_proportion,cluster.test.result
3370,OBSCN,21,0.000387125,8.372589e-08
3272,NOTCH1,11,0.000134817,4.622681e-05
4524,SLC22A10,6,0.000032100,2.575925e-04
3118,MYOM2,8,0.000071900,2.591620e-04
3439,OR4D9,4,0.000013600,2.458870e-03
2527,KLC3,5,0.000028800,3.680924e-03
5011,TJP3,6,0.000048100,3.892065e-03
873,CDH23,9,0.000144621,9.554264e-03
2785,LSP1,2,0.000001930,1.227944e-02
730,CAPN9,5,0.000035600,1.246807e-02
